In [ ]:

# 0 不懂
# 1 转90
# 2 翻转

findDir=[
# "typical_building_colorful_cottage", 
# "typical_vehicle_pirate_ship", 0
# "typical_building_building",
# "typical_creature_dragon",
# "typical_creature_furry", 0 
# "typical_building_castle",
# "ten1o_A_floating_miniature_diorama_of_a_Suzhou_garden_pavilion__04a6bc22-15a0-49ac-b141-b40abb18statckHeightf68", 0
"ten1o_A_miniature__of_a_classical_Suzhou_garden__perched_on_a_f_71b11ddstatckHeight-fa2a-statckHeighte3c-ae67-9ae232836644",
# "ten1o_A_miniature_floating_Suzhou_garden__with_a_small_elegant__29ecf1fe-5a57-42c2-89ec-1b8261d22a63" , 
]

In [ ]:
import numpy as np
import os
from pathlib import Path
from tqdm import tqdm
import subprocess

folderPath = "🐶VDB/"
directories = [d for d in os.listdir(folderPath) if os.path.isdir(os.path.join(folderPath, d))]

for dir in tqdm(directories):
    if dir not in findDir:
        continue
    # if Path(os.path.join(folderPath, dir , "preview-view1.mp4") ).exists():
    #     continue
    print(dir)
    datapath = os.path.join(folderPath, dir , "array/allData.npy") 
    big_tensor =np.load(datapath) # 形状 (N, 273, 944, 4)，N 是文件数量
    # rotated_tensor = np.transpose(big_tensor, (2, 1, 0, 3))
    # # 需要翻转高度轴以保持正确的空间关系
    # rotated_tensor = np.flip(rotated_tensor, axis=2)
    # big_tensor=big_tensor[:,:,::2,:]
    
    print(big_tensor.shape)



    # 提取 r, g, b, alpha 通道
    r = big_tensor[..., 0]  # 取第 1 通道
    g = big_tensor[..., 1]  # 取第 2 通道
    b = big_tensor[..., 2]  # 取第 3 通道
    alpha = big_tensor[..., 3]  # 取第 4 通道


    # 假设 alpha 是一个形状为 (depth, height, width) 的3D数组
    # 初始化3D bbox的范围，使用极大值和极小值确保首次比较能正确更新
    x_min, y_min, z_min = float('inf'), float('inf'), float('inf')
    x_max, y_max, z_max = -float('inf'), -float('inf'), -float('inf')

    for i in range(len(alpha)):
        coordinates = np.where(alpha[i] > .8)
        if coordinates[0].size > 0:
            # 计算当前层的bbox
            current_bbox = [
                coordinates[0].min(), # x_min
                coordinates[1].min(), # y_min
                i,                    # z_min for the current layer
                coordinates[0].max(), # x_max
                coordinates[1].max(), # y_max
                i                     # z_max for the current layer
            ]
            
            # 更新整体3D bbox
            x_min = min(x_min, current_bbox[0])
            y_min = min(y_min, current_bbox[1])
            z_min = min(z_min, current_bbox[2])
            x_max = max(x_max, current_bbox[3])
            y_max = max(y_max, current_bbox[4])
            z_max = max(z_max, current_bbox[5])

    # 输出最终的3D bbox
    if x_min != float('inf') and x_max != -float('inf'):
        print("3D Bounding Box: [x_min={}, y_min={}, z_min={}, x_max={}, y_max={}, z_max={}]".format(x_min, y_min, z_min, x_max, y_max, z_max))
    else:
        print("No elements satisfy the condition across all layers.")
        
    #按照3D bbox裁剪big_tensor
    big_tensor = big_tensor[z_min:z_max+1 , x_min:x_max+1 , y_min:y_max+1]
        
        
    expandRatio=.38
    xExpand=int((x_max-x_min)*(expandRatio))
    yExpand = int((y_max - y_min) * (expandRatio+0.2))
    zExpand = int((z_max - z_min) * expandRatio)

    # 计算扩展后的张量形状
    expanded_shape = (
        big_tensor.shape[0] + zExpand,  # z轴的扩展
        big_tensor.shape[1] + xExpand,  
        big_tensor.shape[2] + yExpand , 
        4
    )

    # 创建一个形状为expanded_shape的全零张量
    big_tensor_expanded = np.zeros(expanded_shape)

    # 计算原始big_tensor在扩展张量中的位置
    z_offset = zExpand // 2
    y_offset = yExpand // 2
    x_offset = xExpand // 2

    # 将原始big_tensor复制到big_tensor_expanded的中心

    big_tensor_expanded[z_offset:z_offset + big_tensor.shape[0],
                        0: big_tensor.shape[1],
                        y_offset:y_offset+ big_tensor.shape[2]] = big_tensor
    
    # 左边顶到底
    # big_tensor_expanded[z_offset:z_offset + big_tensor.shape[0],
                        # x_offset: x_offset+big_tensor.shape[1],
                        # 0: big_tensor.shape[2]] = big_tensor

    # 打印结果形状确认
    print("Expanded tensor shape:", big_tensor_expanded.shape)

    # 取出datapath的目录
    datapath = os.path.dirname(datapath)
    datapath=os.path.join(datapath,"..", "cut")
    os.makedirs(datapath, exist_ok=True)
    np.save(os.path.join(datapath, "allData.npy"), big_tensor_expanded)

    # os.system(f"python /media/vrlab/rabbit/print3dingp/print_volume/preview_volume.py --input_folder {datapath} --onlyOneView ")

    subprocess.run(
        ["python", "/media/vrlab/rabbit/print3dingp/print_volume/preview_volume.py", "--input_folder", datapath, "--onlyOneView"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )

  0%|          | 0/28 [00:00<?, ?it/s]

ten1o_A_miniature__of_a_classical_Suzhou_garden__perched_on_a_f_71b11dd4-fa2a-4e3c-ae67-9ae232836644
(1428, 236, 472, 4)
3D Bounding Box: [x_min=35, y_min=4, z_min=134, x_max=201, y_max=471, z_max=1283]
Expanded tensor shape: (1586, 230, 738, 4)


100%|██████████| 28/28 [00:58<00:00,  2.09s/it]
